# Calculating longitudinal profiles of Pacific temp
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

#### Mechanics
Read dependencies

In [1]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from matplotlib import rcParams
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.io import loadmat
from scipy import signal

Model names

In [2]:
modelnames_fig = ['CCSM3 abrupt 2x','CCSM3 abrupt 4x','CCSM3 abrupt 8x', \
    'CESM1.0.4 abrupt 2x','CESM1.0.4 abrupt 4x','CESM1.0.4 abrupt 8x', 'CNRM-CM6.1 abrupt4x', \
    'GFDL-CM3 1pct 2x','GFDL-ESM2M 1pct 2x','GISS-E2-R 1pct 4x', \
    'GISS-E2-R abrupt 4x','HadCM3L abrupt 2x','HadCM3L abrupt 4x', \
    'HadCM3L abrupt 6x','HadCM3L abrupt 8x','IPSL-CM5A-LR abrupt 4x', \
    'MIROC3.2 1pct 2x','MIROC3.2 1pct 4x','MPIESM-1.2 abrupt 2x', \
    'MPIESM-1.2 abrupt 4x','MPIESM-1.2 abrupt 8x']

modelnames_file = ['CCSM3_abrupt2x','CCSM3_abrupt4x','CCSM3_abrupt8x', \
    'CESM104_abrupt2x','CESM104_abrupt4x','CESM104_abrupt8x', \
    'CNRMCM61_abrupt4x','GFDLCM3_1pct2x','GFDLESM2M_1pct2x','GISSE2R_1pct4x', \
    'GISSE2R_abrupt4x','HadCM3L_abrupt2x','HadCM3L_abrupt4x', \
    'HadCM3L_abrupt6x','HadCM3L_abrupt8x','IPSLCM5A_abrupt4x', \
    'MIROC32_1pct2x','MIROC32_1pct4x','MPIESM12_abrupt2x', \
    'MPIESM12_abrupt4x','MPIESM12_abrupt8x']

Data location

In [3]:
loc_tas = "~/" # change if using raw data
loc_out = "../Data/Pacific_Profiles/"

Function for listing files

In [4]:
def list_files(directory, extension):
    
    # Credit to: http://www.martinbroadhurst.com/listing-all-files-in-a-directory-with-a-certain-extension-in-python.html
    # Requires os.listdir
    
    return (f for f in os.listdir(directory) if f.endswith(extension))

Area info

In [5]:
lat_min = -5
lat_max = 5
lon_min = 130
lon_max = 280

#### Analysis

Loop through simulations, calculate profiles, output

In [6]:
for i in np.arange(0,len(modelnames_file),1):
    
    model, exp = modelnames_file[i].split("_")
    print(modelnames_file[i])
    
    fname_exp_start = "tas_mon_"+modelnames_file[i]
    fname_control_start = "tas_mon_"+model+"_control"
    
    fname_exp = [f for f in os.listdir(loc_tas) if f.startswith(fname_exp_start)][0]
    fname_control = [f for f in os.listdir(loc_tas) if f.startswith(fname_control_start)][0]
    
    n1, n2, n3, n4, year_exp_nc = fname_exp.split("_")
    n1, n2, n3, n4, year_control_nc = fname_control.split("_")
    year_exp, nc = year_exp_nc.split(".")
    year_control, nc = year_control_nc.split(".")
    
    # In this script we're repeatedly doing the calculations for the control, for each experiment
    # That is, this loop for CESM abrupt2x and abrupt4x does the exact same calculation on the
    # CESM control and re-writes the file
    # but I'm too lazy to be more precise about it
    
    model_decode_times_list = ["MIROC32","MPIESM12"]
    
    if model in model_decode_times_list:
        tas_exp_global = xr.DataArray(xr.open_dataset(loc_tas+fname_exp,decode_times=False).data_vars["tas"]) 
        tas_control_global = xr.DataArray(xr.open_dataset(loc_tas+fname_control,decode_times=False).data_vars["tas"]) 
    else:
        tas_exp_global = xr.DataArray(xr.open_dataset(loc_tas+fname_exp).data_vars["tas"])
        tas_control_global = xr.DataArray(xr.open_dataset(loc_tas+fname_control).data_vars["tas"])
            
    if model == "HadCM3L":
        lat_tas = tas_exp_global.coords["latitude_1"]
        lon_tas = tas_exp_global.coords["longitude_1"]
    else:
        lat_tas = tas_exp_global.coords["lat"]
        lon_tas = tas_exp_global.coords["lon"]
            
    if lat_tas[0] > lat_tas[len(lat_tas.values)-1]:
        profile_exp_raw = tas_exp_global.loc[:,lat_max:lat_min,lon_min:lon_max].mean(axis=1)
        profile_control_raw = tas_control_global.loc[:,lat_max:lat_min,lon_min:lon_max].mean(axis=1)
    else:
        profile_exp_raw = tas_exp_global.loc[:,lat_min:lat_max,lon_min:lon_max].mean(axis=1)
        profile_control_raw = tas_control_global.loc[:,lat_min:lat_max,lon_min:lon_max].mean(axis=1)
    lon_profile = lon_tas.loc[lon_min:lon_max]
    
    #index_exp_raw = tas_exp.mean(dim=["lat","lon"])
    #index_control_raw = tas_exp.mean(dim=["lat","lon"])
        
    # attach time arrays to allow us to calculate anomalies
    n_months_c = profile_control_raw.shape[0]
    n_months_f = profile_exp_raw.shape[0]
    time_exp = xr.cftime_range(start='0001', periods=n_months_f, freq='M')
    time_control = xr.cftime_range(start='0001', periods=n_months_c, freq='M')
            
    profile_exp_time = xr.DataArray(profile_exp_raw.values,coords=[time_exp,lon_profile],dims=["time","lon"])
    profile_control_time = xr.DataArray(profile_control_raw.values,coords=[time_control,lon_profile],dims=["time","lon"])
    
    # Write out data
    profile_exp_time.name = "profile"
    profile_exp_time.attrs["creation_date"] = str(datetime.datetime.now())
    profile_exp_time.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
    profile_exp_time.attrs["data_description"] = "longitudinal temperature profile, "+str(lat_min)+"-"+str(lat_max)+", "+str(lon_min)+"-"+str(lon_max)
    profile_exp_time.attrs["created_from"] = "Calculate_Pacific_Profiles.ipynb"
        
    fname_out_exp = loc_out+"pacific_lon_profile_"+model+"_"+exp+".nc"
    profile_exp_time.to_netcdf(fname_out_exp,mode="w")
    print(fname_out_exp)
        
        
    # write out raw index from control
    profile_control_time.name = "profile"
    profile_control_time.attrs["creation_date"] = str(datetime.datetime.now())
    profile_control_time.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
    profile_control_time.attrs["data_description"] = "longitudinal temperature profile, "+str(lat_min)+"-"+str(lat_max)+", "+str(lon_min)+"-"+str(lon_max)
    profile_control_time.attrs["created_from"] = "Calculate_Pacific_Profiles.ipynb"
        
    fname_out_control = loc_out+"pacific_lon_profile_"+model+"_control.nc"
    profile_control_time.to_netcdf(fname_out_control,mode="w")
    print(fname_out_control)

CCSM3_abrupt2x
../Data/Pacific_Profiles/pacific_lon_profile_CCSM3_abrupt2x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CCSM3_control.nc
CCSM3_abrupt4x
../Data/Pacific_Profiles/pacific_lon_profile_CCSM3_abrupt4x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CCSM3_control.nc
CCSM3_abrupt8x
../Data/Pacific_Profiles/pacific_lon_profile_CCSM3_abrupt8x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CCSM3_control.nc
CESM104_abrupt2x
../Data/Pacific_Profiles/pacific_lon_profile_CESM104_abrupt2x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CESM104_control.nc
CESM104_abrupt4x
../Data/Pacific_Profiles/pacific_lon_profile_CESM104_abrupt4x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CESM104_control.nc
CESM104_abrupt8x
../Data/Pacific_Profiles/pacific_lon_profile_CESM104_abrupt8x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CESM104_control.nc
CNRMCM61_abrupt4x


/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, or

../Data/Pacific_Profiles/pacific_lon_profile_CNRMCM61_abrupt4x.nc
../Data/Pacific_Profiles/pacific_lon_profile_CNRMCM61_control.nc
GFDLCM3_1pct2x


/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


../Data/Pacific_Profiles/pacific_lon_profile_GFDLCM3_1pct2x.nc
../Data/Pacific_Profiles/pacific_lon_profile_GFDLCM3_control.nc
GFDLESM2M_1pct2x
../Data/Pacific_Profiles/pacific_lon_profile_GFDLESM2M_1pct2x.nc
../Data/Pacific_Profiles/pacific_lon_profile_GFDLESM2M_control.nc
GISSE2R_1pct4x
../Data/Pacific_Profiles/pacific_lon_profile_GISSE2R_1pct4x.nc
../Data/Pacific_Profiles/pacific_lon_profile_GISSE2R_control.nc
GISSE2R_abrupt4x
../Data/Pacific_Profiles/pacific_lon_profile_GISSE2R_abrupt4x.nc
../Data/Pacific_Profiles/pacific_lon_profile_GISSE2R_control.nc
HadCM3L_abrupt2x
../Data/Pacific_Profiles/pacific_lon_profile_HadCM3L_abrupt2x.nc
../Data/Pacific_Profiles/pacific_lon_profile_HadCM3L_control.nc
HadCM3L_abrupt4x
../Data/Pacific_Profiles/pacific_lon_profile_HadCM3L_abrupt4x.nc
../Data/Pacific_Profiles/pacific_lon_profile_HadCM3L_control.nc
HadCM3L_abrupt6x
../Data/Pacific_Profiles/pacific_lon_profile_HadCM3L_abrupt6x.nc
../Data/Pacific_Profiles/pacific_lon_profile_HadCM3L_control.nc